# APPLICATION  D'UNE MCMC pour la SIMULATION A POSTERIORI

Donnees X (debit de riviere) ~ Loi de GUMBEL (mu,lambda)  
A priori sur (mu,lambda) = melange de lois Gamma d'hyperparametres (m, alpha, xe.alpha)  
avec :  
m = taille d'echantillon fictif (force de l'expertise)  
xe_alpha = quantile predictif (sur X, donc positif) a priori de seuil alpha (donne par l'expert)  
alpha    = seuil de xe.alpha, compris dans [0,1]  (donne par l'expert)  
lambda_e = moyenne a priori de lambda (donnee par l'expert)

In [1]:
%matplotlib notebook
from math import floor
from scipy import log, pi, exp, sqrt
from scipy import __version__ as sci_version
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.special import loggamma
from numpy.random import normal, uniform, gamma
from statistics import mean
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels import __version__ as stm_version
from matplotlib import __version__ as mpl_version
from pandas import __version__ as pd_version
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
print(
      "scipy version " +  sci_version + 
      "\nnumpy version " + np.__version__ +
      "\nmatplotlib version " + mpl_version +
      "\nstatsmodels version " + stm_version +
      "\npandas version " + pd_version
     )

scipy version 1.3.1
numpy version 1.16.0
matplotlib version 3.1.1
statsmodels version 0.11.0
pandas version 0.25.3


## Jeu de donnees 

30 mesures de débit

In [3]:
data = [ 1306, 1235, 1830, 2442, 1128, 3330, 1530, 3192, 2647, 238,  
        706, 1903, 1594,  935, 1100, 2204, 1366, 1629,  522,  642, 1173, 
        424, 1837, 1391, 789,  383, 1858, 917, 1084, 1026]

In [4]:
m = 1
xe_alpha=2000
alpha=0.5
lambda_e=1/2000

##  Algorithme de Metropolis-Hastings-within-Gibbs 

### Densité a priori sur la moyenne

A priori :  
$\lambda \sim Gamma(m, m/\lambda_e)$ en paramètrisation $Gamma(shape, rate)$, $rate = 1/scale $  
$mu \sim Gamma(m, b_m(\lambda))$  en paramètrisation $Gamma(shape, rate)$  
avec $b_m(\lambda) = exp(- \lambda x_{e,\alpha}) / (\alpha^{-1/m} - 1)$

In [5]:
def get_mean_rate_param(l, m, xe_alpha, alpha) :
    """
        Fonction donnant l'hyperparametre d'echelle de la loi a priori de mu
        sachant lambda, m, xe_alpha et alpha
    """
    return exp(- l * xe_alpha) / (alpha ** (- 1 / m) - 1)

def sample_from_prior(n, m, xe_alpha, alpha, lambda_e) :
    """
         Simulation de (mu,lambda) a priori
    """
    l = gamma(shape = m, scale = lambda_e / m, size = n)
    scale_prior = get_mean_rate_param(l, m, xe_alpha, alpha)
    mu = gamma(shape = m, scale = 1 / scale_prior, size = n)
    return {"lambda" : l, "mu" : mu}

### Log-densité a priori sur lambda

In [6]:
def lambda_prior_density(l, lambda_e, m) :
    # A COMPLETER

### Log-densite a posteriori de lambda 

A posteriori :  
$\pi(\lambda | x_n) = \gamma(\lambda) Gamma(m + n , m/\lambda_e + n\bar{x}_n)$  
avec 
$\bar{x}_n = \frac{1}{n} \sum_{i=1}^n x_i$,  
$\gamma(\lambda) \propto \frac{b^m_m(\lambda)}{(b_m(\lambda) + \bar{b}_{x_n}(\lambda))^{m + n}}$  
et $\bar{b}_{x_n} = \sum_{i = 1}^n exp(-\lambda x_i)$

In [7]:
def compute_informed_parameters(l, uncensored_data = np.array([]), censored_data = np.array([])) :  
    
   # A COMPLETER

def lambda_log_posterior(l, m, xe_alpha, alpha, lambda_e, uncensored_data, censored_data) :
    
 # A COMPLETER

### Simulation conditionnelle de Gibbs de mu sachant lambda a posteriori

In [8]:
def sample_mu_from_posterior(l, m, xe_alpha, alpha, lambda_e,
                           uncensored_data, censored_data) :
    
    # A COMPLETER

### Estimateurs du maximum de vraisemblance de la loi de Gumbel

 Estimation rapide des estiamteurs du max de vraisemblance de Gumbel sur des donnees non censurees (via un algorithme de simplexe), utilisés pour initialiser les valeurs de mu et lambda.

In [9]:
def log_density_Gumbel(data, mu, l) :
    return log(l) + log(mu) - l * data - mu * exp(- l* data)

def log_likelihood_Gumbel(uncensored_data, censored_data, mu, l) :
    
    ll = 0
    
    if uncensored_data.size > 0 :   
        ll += np.sum(log_density_Gumbel(uncensored_data, mu, l))

    if censored_data.size > 0 :   
        ll += np.sum(- mu * exp(- l* censored_data))

    return ll


def MLE_Gumbel(data, plotting= True) :
    
    # log-vraisemblance negative a minimiser
    # A COMPLETER
        
    return MLE_param

### Initialisation de lambda

In [10]:
def initialize_lambda(n_chains, data) :
    MLE = MLE_Gumbel(data, plotting = False)
    return uniform(low = 0, high = 1.5 * MLE["lambda"], size = n_chains)

### Selection de la distribution instrumentale

In [11]:
def make_instrumental_distribution(nb_chains, option, m, xe_alpha, alpha, lambda_e, data) :
    """
        Fonction générant les fonction des log_densite et d'echantillonnage selon l'option selectionnee.
    """
    if option == 1 : 
        
        print("Loi instrumentale choisie : loi a priori\n")
        
        def log_density(x,y) :
            return m * (log(m) - log(lambda_e)) - loggamma(m) + (m - 1) * log(x) - m * x / lambda_e
        
        def sampling(n,x) :
            sample = gamma(shape = m, scale = lambda_e / m, size = (n * nb_chains))
            return np.ndarray(shape =(n,nb_chains), buffer = sample)
    
    
    if option == 2 : 
        
        print("Loi instrumentale choisie : loi gamma semblant proche de l'a posteriori sur lambda\n")
        n = len(data)
        xn = np.mean(data)
        
        def log_density(x,y) :
            return (m + n) * log(m / lambda_e + n * xn) - loggamma(m + n) + (m + n - 1) * log(x) - x * (m / lambda_e + n * xn)
        
        def sampling(n,x) :
            sample = gamma(shape = n + m, scale = 1 / (m / lambda_e + n * xn), size = n * nb_chains)
            return np.ndarray(shape =(n,nb_chains), buffer = sample)

        
    if option == 3 :
        
        print("Loi instrumentale choisie : loi normale de moyenne la valeur courante et de coeff. de variation = 5 %")
        coeff_variation = 0.05
        
        def log_density(x, y) :
            sigma = abs(coeff_variation * y)
            return -0.5 * log(2 * pi) - log(sigma) - (x-y) ** 2 / (2 * (sigma ** 2))
        
        def sampling(n, x) :
            sigma = abs(coeff_variation * x)
            sample = abs(normal(loc = x, scale = sigma, size = n * nb_chains))
            return np.ndarray(shape =(n,nb_chains), buffer = sample)
    
    
    if option == 4 :
        
        print("Loi instrumentale choisie : loi normale de moyenne la valeur courante et de coeff. de variation = 25 %")
        coeff_variation = 0.25
        
        def log_density(x, y) :
            sigma = abs(coeff_variation * y)
            return -0.5 * log(2 * pi) - log(sigma) - (x-y) ** 2 / (2 * (sigma ** 2))
        
        def sampling(n, x) :
            sigma = abs(coeff_variation * x)
            sample = abs(normal(loc = x, scale = sigma, size = n * nb_chains))
            return np.ndarray(shape =(n,nb_chains), buffer = sample)
        
    if option == 5 :
        
        print("Loi instrumentale choisie : loi normale de moyenne la valeur courante et de coeff. de variation = 50 %")
        coeff_variation = 0.5
        
        def log_density(x, y) :
            sigma = abs(coeff_variation * y)
            return -0.5 * log(2 * pi) - log(sigma) - (x-y) ** 2 / (2 * (sigma ** 2))
        
        def sampling(n, x) :
            sigma = abs(coeff_variation * x)
            sample = abs(normal(loc = x, scale = sigma, size = n * nb_chains))
            return np.ndarray(shape =(n,nb_chains), buffer = sample)
        
    return log_density, sampling

### Construction de la fonction de densité a posteriori pour lambda

In [12]:
 def make_log_posterior_density(m, xe_alpha, alpha, lambda_e, uncensored_data, censored_data) :
        
        def log_posterior_density(l) :
            return lambda_log_posterior(l, m, xe_alpha, alpha, lambda_e, uncensored_data, censored_data)
        return log_posterior_density

### Diagnostic de l'échantillonnage

####                 Statistique de Brooks-Gelman

calculee sur un nombre nb_chains de chaines MCMC paralleles pour le parametre theta (matrix Nsim x nb_chains)

In [13]:
def Brooks_Gelman(theta, pro = 0.9) :
    
 # A COMPLETER


### Echantillonnage

In [14]:
def metropolis_hastings_sampling(lambda_old, sampling, log_density, log_posterior_density) :
    
    # A COMPLETER
    
    return sample

### Graphes

In [15]:
# A COMPLETER

### Boucle MCMC complète

In [16]:
def metropolis_hastings_within_gibbs(data, option = 1, N = 1000, burn_in = 5000, nb_chains = 3,
                                     m = 1, xe_alpha = 2000, alpha = 0.5, lambda_e = 1/2000, 
                                     pause = range(1,101), plotting= range(1, 10)) :
    
    # A COMPLETER
